# GPU for pytorch

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name()

'Tesla T4'

In [5]:
torch.cuda.memory_allocated()

0

In [6]:
torch.cuda.memory_cached()

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


0

In [7]:
a = torch.FloatTensor([1.0, 2.0])

In [8]:
a

tensor([1., 2.])

In [9]:
a.device

device(type='cpu')

## Use GPU and CUDA

In [10]:
a = torch.FloatTensor([1.0, 2.0]).cuda()

In [11]:
a.device

device(type='cuda', index=0)

In [12]:
torch.cuda.memory_allocated()

512

## Imports

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Creating Model

In [37]:
class Model(nn.Module):

  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)    # input leayer
    self.fc2 = nn.Linear(h1, h2)             # hidden layer
    self.out = nn.Linear(h2, out_features)   # output layer

  def forward(self, X):
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.out(X)
    return X

In [38]:
model = Model()

In [39]:
# our model is not on gpu
next(model.parameters()).is_cuda

False

### store on gpu

In [40]:
gpumodel = model.cuda()

In [41]:
next(gpumodel.parameters()).is_cuda

True

#### IRIS Dataset

In [42]:
from sklearn.datasets import load_iris

In [43]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns = iris.feature_names)

In [44]:
df['target'] = iris.target

In [45]:
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [46]:
X = df.drop("target", axis=1).values

In [47]:
y = df["target"].values

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
# gpu and cuda
X_train = torch.Tensor(X_train).cuda()

In [50]:
X_test = torch.Tensor(X_test).cuda()

In [51]:
print(X_train.dtype)

torch.float32


In [52]:
y_train = torch.Tensor(y_train).cuda()

In [53]:
y_test = torch.Tensor(y_test).cuda()

In [54]:
train_loader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
test_loader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [55]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## Train Model

In [56]:
import time
start_time = time.time()
epochs = 100
losses = []

for i in range(epochs):

  y_pred = gpumodel.forward(X_train)

  y_train = y_train.long()
  loss = criterion(y_pred, y_train)
  losses.append(loss)

  if i%10 == 0:
    print(f"Epoch: {i} Loss: {loss.item()}")

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()



end = time.time() - start_time
print(f"TOTAL TIME: ", {end})


Epoch: 0 Loss: 1.3868739604949951
Epoch: 10 Loss: 0.9827079772949219
Epoch: 20 Loss: 0.7703072428703308
Epoch: 30 Loss: 0.5199500918388367
Epoch: 40 Loss: 0.34135475754737854
Epoch: 50 Loss: 0.22576814889907837
Epoch: 60 Loss: 0.14724616706371307
Epoch: 70 Loss: 0.10403325408697128
Epoch: 80 Loss: 0.08314085751771927
Epoch: 90 Loss: 0.07274985313415527
TOTAL TIME:  {0.1324176788330078}


In [57]:
x = torch.tensor([1,2])

In [58]:
x.dtype

torch.int64

## Whole Dataset

In [59]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = gpumodel.forward(data)
    print(f"{i+1:2}. {str(y_val):38} {y_test[i]}")
    if y_val.argmax().item() == y_test[i]:
      correct += 1

print(f"\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct")

 1. tensor([-3.3326,  3.5881, -0.4133], device='cuda:0') 1.0
 2. tensor([  6.9964,   0.9022, -21.0897], device='cuda:0') 0.0
 3. tensor([-8.9299,  4.0037, 13.8985], device='cuda:0') 2.0
 4. tensor([-3.6043,  3.4670,  0.6857], device='cuda:0') 1.0
 5. tensor([-3.8896,  3.8072,  0.5095], device='cuda:0') 1.0
 6. tensor([  6.3214,   0.8818, -19.1658], device='cuda:0') 0.0
 7. tensor([-1.4910,  2.9749, -3.7564], device='cuda:0') 1.0
 8. tensor([-6.0531,  3.6344,  6.9918], device='cuda:0') 2.0
 9. tensor([-4.6901,  3.3275,  4.0834], device='cuda:0') 1.0
10. tensor([-2.1404,  3.2193, -2.6626], device='cuda:0') 1.0
11. tensor([-5.2140,  3.6080,  4.7415], device='cuda:0') 2.0
12. tensor([  6.1357,   0.7889, -18.4252], device='cuda:0') 0.0
13. tensor([  7.0198,   0.8464, -21.0193], device='cuda:0') 0.0
14. tensor([  6.2284,   0.8060, -18.7242], device='cuda:0') 0.0
15. tensor([  6.9184,   0.7826, -20.5838], device='cuda:0') 0.0
16. tensor([-3.4874,  3.6870, -0.2600], device='cuda:0') 1.0
17. te